In [1]:
import gymnasium as gym
from gridworld.envs import GridWorldEnv, SiblingGridWorldEnv
from gymnasium.wrappers import FlattenObservation
from gridworld.wrappers import RelativePosition, InvertedReward
from gymnasium.envs.registration import register

<frozen importlib._bootstrap>:241: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


In [2]:
def sibling():
    env = SiblingGridWorldEnv()
    env = InvertedReward(env)
    return env

register(
    id='SiblingGridWorld-v0',
    entry_point=sibling,
    max_episode_steps=300,
)

In [3]:
env = gym.make('SiblingGridWorld-v0')
print(env.reset())
# wrapped_env = InvertedReward(env)
# print(wrapped_env.reset())

({'agent': array([0, 4]), 'target': array([3, 4])}, {'distance': 3.0})


In [4]:
env = env.unwrapped

In [5]:
env.render_mode = 'human'
env.reset()

({'agent': array([1, 4]), 'target': array([4, 3])}, {'distance': 4.0})

In [9]:
obs, rew, done, trunc, info = env.step(0)
obs, rew, done, trunc, info

({'agent': array([4, 3]), 'target': array([4, 3])},
 1,
 True,
 False,
 {'distance': 0.0})

In [10]:
env.close()

: 

In [7]:
import numpy as np
from itertools import permutations

In [36]:
true_map = np.array(
    [[1, 0], [0, -1], [-1, 0], [0, 1]]
)

In [37]:
np.array(list(permutations(true_map)))[1]

array([[ 1,  0],
       [ 0, -1],
       [ 0,  1],
       [-1,  0]])

In [38]:
d = dict(zip(range(24), np.array(list(permutations(true_map)))))

In [39]:
np.array_equal(true_map, d[0])

True